<!-- -*- mode: markdown; coding: utf-8; fill-column: 60; ispell-dictionary: "english" -*- -->

<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<link rel="stylesheet" href="style.css">


# EDAF75 - lab 2: Testing the database

As usual we have to tell Jupyter to allow SQL:

In [1]:
%load_ext sql

And then we import our movie database

In [2]:
%sql sqlite:///movies.sqlite

We want to make sure that SQLite3 really checks our foreign
key constraints -- to do that, we run:

In [3]:
%%sql
PRAGMA foreign_keys=ON;

 * sqlite:///movies.sqlite
Done.


[]

Now write SQL code for the following tasks:


+ Show the names of all movies.

In [8]:
%%sql
SELECT * 
FROM screening

 * sqlite:///movies.sqlite
Done.


start_time,screening_id,theater_name,IMDB_key


+ Show the performance dates for one of the movies.

In [7]:
%%sql
SELECT start_time
FROM screening
WHERE IMDB_key='tt10332588'

 * sqlite:///movies.sqlite
Done.


start_time


## + Show all data concerning performances at a given theatere
  on a given date.

In [13]:
%%sql
SELECT * 
FROM screening
WHERE theater_name='Råsunda'

 * sqlite:///movies.sqlite
Done.


start_time,screening_id,theater_name,IMDB_key
2024-02-20 18:00:00,c71ee5f1e1043ad51ed68c9a3e49c372,Råsunda,tt0381707


+ List all customers

In [14]:
%%sql
SELECT *
FROM customer

 * sqlite:///movies.sqlite
Done.


username,full_name,password
LuddeFnutt,Ludwig Cederberg,VonNeumann12
SaraLightSaber,Sara Saber,HACKe13
Gabbecito,Gabriel Wåhlin,VAlfalk42


+ List all tickets

In [20]:
%%sql
SELECT *
FROM ticket

 * sqlite:///movies.sqlite
Done.


ticket_id,username,screening_id
5c4883508cea3f2f87fe50531b06bc82,LuddeFnutt,c71ee5f1e1043ad51ed68c9a3e49c372


+ Create a new ticket to some performance (i.e., insert a
  new row in your table of tickets).

In [19]:
%%sql
INSERT INTO ticket(ticket_id, username, screening_id)
VALUES (lower(hex(randomblob(16))), "LuddeFnutt", "c71ee5f1e1043ad51ed68c9a3e49c372");

 * sqlite:///movies.sqlite
1 rows affected.


[]

In newer versions of SQLite (since version 3.35, released
  in March 2021), and in
  [PostgreSQL](https://www.postgresql.org/docs/current/sql-insert.html),
  we can get any value generated during an insert using the
  `INSERT...-RETURNING` statement:

~~~{.sql}
INSERT
INTO       students
VALUES     ('Amy', 3.9, 1200)
RETURNING  s_id
~~~


which would return the generated `s_id` for the new
  student.

  If your SQLite version is older than 3.35, and you can't
  upgrade, you can instead use the following idea: each row
  in a SQLite3 table has a `rowid` attribute, it is a unique
  integer which essentially tells in which order the rows
  were inserted, and it's not displayed in queries unless we
  ask for it. SQLite3 also have a function,
  `last_insert_rowid()`, which returns the `rowid` of the
  last inserted row of a table, so we can see the `s_id` of
  the most recently inserted student with the following
  query:

~~~{.sql}
SELECT s_id
FROM   students
WHERE  rowid = last_insert_rowid();
~~~


Now, check what ticket number we got for the ticket we
  created above (it should be the same as the ticket id,
  which should be a `randomblob`):

In [21]:
%%sql
SELECT ticket_id
FROM ticket
WHERE rowid = last_insert_rowid()

 * sqlite:///movies.sqlite
Done.


ticket_id
5c4883508cea3f2f87fe50531b06bc82


+ Try to insert two movie theaters with the same name (this
  should fail).

In [23]:
%%sql
INSERT INTO theater
VALUES ("Råsunda", 100)

 * sqlite:///movies.sqlite
(sqlite3.IntegrityError) UNIQUE constraint failed: theater.name
[SQL: INSERT INTO theater
VALUES ("Råsunda", 100)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


+ Try to insert a performance where the theater doesn’t
  exist in the database (this should fail).

In [24]:
%%sql
INSERT INTO screening
VALUES('2024-02-20 18:00:00', lower(hex(randomblob(16))), "Rås", "tt0381707")

 * sqlite:///movies.sqlite
(sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT INTO screening
VALUES('2024-02-20 18:00:00', lower(hex(randomblob(16))), "Rås", "tt0381707")]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


+ Create a ticket where either the user or the performance
  doesn’t exist (this should fail).

In [25]:
%%sql
INSERT INTO ticket
VALUES (lower(hex(randomblob(16))), "RubensKatt", "c71ee5f1e1043ad51ed68c9a3e49c372")

 * sqlite:///movies.sqlite
(sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT INTO ticket
VALUES (lower(hex(randomblob(16))), "RubensKatt", "c71ee5f1e1043ad51ed68c9a3e49c372")]
(Background on this error at: https://sqlalche.me/e/20/gkpj)
